# Sentiment Analysis of Twitter Posts
<!-- Notebook name goes here -->
<center><b>Notebook: Data Description, Cleaning, Exploratory Data Analysis, and Preprocessing</b></center>
<br>

**By**: Stephen Borja, Justin Ching, Erin Chua, and Zhean Ganituen.

**Dataset**: Hussein, S. (2021). Twitter Sentiments Dataset [Dataset]. Mendeley. https://doi.org/10.17632/Z9ZW7NT5H2.1

**Motivation**: Every minute, social media users generate a large influx of textual data on live events. Performing sentiment analysis on this data provides a real-time view of public perception, enabling quick insights into the general population’s opinions and reactions.

**Goal**: By the end of the project, our goal is to create and compare supervised learning algorithms for sentiment analysis.

### **Dataset Description**

The Twitter Sentiments Dataset is a dataset that contains nearly 163k tweets from Twitter. The time period of when these were collected is unknown, but it was published to Mendeley Data on May 14, 2021 by Sherif Hussein of Mansoura University.

Tweets were extracted using the Twitter API, but the specifics of how the tweets were selected are unmentioned. The tweets are mostly English with a mix of some Hindi words for code-switching<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1). All of them seem to be talking about the political state of India. Most tweets mention Narendra Modi, the current Prime Minister of India.

Each tweet was assigned a label using TextBlob's sentiment analysis (El‑Demerdash, Hussein, & Zaki, 2021), which assigns labels automatically.

What each row and column represents: `each row represents one tweet.`
Number of observations: `162,980`

---
**References**
1. Alaa A. El-Demerdash, J. F. W. Z., Sherif E. Hussein. (2022). Course Evaluation Based on Deep Learning and SSA Hyperparameters Optimization. Computers, Materials & Continua, 71(1), 941–959. doi:10.32604/cmc.2022.021839

---

<a name="cite_note-1"></a>1. [^](#cite_ref-1) Code-switching is the practice of alternating between two languages $L_1$ (the native language) and $L_2$ (the source language) in a conversation. In this context, the code-switching is done to appear more casual since the conversation is done via Twitter (now, X). 

Twitter_Data
- **`clean_text`**: The tweet's text
- **`category`**: The tweet's sentiment category

## 1 project set up
Set up here the imports for the projects (ensure these are installed via uv and is part of the environment). Furthermore, load the dataset here. Also load the raw dataset here.

In [53]:
# imports
# no need to import `torch` here since no training occurs yet.
import numpy as np
import pandas as pd

import unicodedata
import re
import string

# Load raw data file
df = pd.read_csv("../data/Twitter_Data.csv")

# At this point, we only want the `clean_text` column
#     Remark. The `clean_text` column (at this point in the project) is a bit of a misnomer.
#     Since, after manual validation and exploration, it is still dirty.
#     Hence, moving forward, the `clean_text` column will be simply referred to as `text`.
text = df["clean_text"]
#text.sample(n=10)    # sanity check

## 2 data cleaning
This section discusses the methodology for data cleaning.

We follow a similar methodology for data cleaning presented in [1]. We start with lowercase converting all entries, whitespace removal, tokenization, stop words removal, punctuation and non-alphabetic character removal, stemming, and lemmatization. For stop words removal, we refer to the English stopwords dataset defined in NLTK and Wolfram Mathematica [2,3].

---
**References**
1. George, M., & Murugesan, R., Dr. (2024). Improving sentiment analysis of financial news headlines using hybrid Word2Vec-TFIDF feature extraction technique. Procedia Computer Science, 244, 1–8. https://doi.org/10.1016/j.procs.2024.10.172
2. Wolfram Language. (2015). DeleteStopwords. Wolfram Language & System Documentation Center. Retrieved from https://reference.wolfram.com/language/ref/DeleteStopwords.html
3. Bird, S., & Loper, E. (2004, July). NLTK: The Natural Language Toolkit. Proceedings of the ACL Interactive Poster and Demonstration Sessions, 214–217. Retrieved from https://aclanthology.org/P04-3031/

In [85]:
# stopwords lut definition
# the code generation script for this lookup table is defined in `make_stopwords_lut.py`.
stopwords_lut = {
    "0o": True,
    "0s": True,
    "3a": True,
    "3b": True,
    "3d": True,
    "6b": True,
    "6o": True,
    "a": True,
    "a1": True,
    "a2": True,
    "a3": True,
    "a4": True,
    "ab": True,
    "able": True,
    "about": True,
    "above": True,
    "abst": True,
    "ac": True,
    "accordance": True,
    "according": True,
    "accordingly": True,
    "across": True,
    "act": True,
    "actually": True,
    "ad": True,
    "added": True,
    "adj": True,
    "ae": True,
    "af": True,
    "affected": True,
    "affecting": True,
    "affects": True,
    "after": True,
    "afterwards": True,
    "ag": True,
    "again": True,
    "against": True,
    "ah": True,
    "ain": True,
    "aint": True,
    "aj": True,
    "al": True,
    "all": True,
    "allow": True,
    "allows": True,
    "almost": True,
    "alone": True,
    "along": True,
    "already": True,
    "also": True,
    "although": True,
    "always": True,
    "am": True,
    "among": True,
    "amongst": True,
    "amoungst": True,
    "amount": True,
    "an": True,
    "and": True,
    "announce": True,
    "another": True,
    "any": True,
    "anybody": True,
    "anyhow": True,
    "anymore": True,
    "anyone": True,
    "anything": True,
    "anyway": True,
    "anyways": True,
    "anywhere": True,
    "ao": True,
    "ap": True,
    "apart": True,
    "apparently": True,
    "appear": True,
    "appreciate": True,
    "appropriate": True,
    "approximately": True,
    "ar": True,
    "are": True,
    "aren": True,
    "arent": True,
    "arise": True,
    "around": True,
    "as": True,
    "aside": True,
    "ask": True,
    "asking": True,
    "associated": True,
    "at": True,
    "au": True,
    "auth": True,
    "av": True,
    "available": True,
    "aw": True,
    "away": True,
    "awfully": True,
    "ax": True,
    "ay": True,
    "az": True,
    "b": True,
    "b1": True,
    "b2": True,
    "b3": True,
    "ba": True,
    "back": True,
    "bc": True,
    "bd": True,
    "be": True,
    "became": True,
    "because": True,
    "become": True,
    "becomes": True,
    "becoming": True,
    "been": True,
    "before": True,
    "beforehand": True,
    "begin": True,
    "beginning": True,
    "beginnings": True,
    "begins": True,
    "behind": True,
    "being": True,
    "believe": True,
    "below": True,
    "beside": True,
    "besides": True,
    "best": True,
    "better": True,
    "between": True,
    "beyond": True,
    "bi": True,
    "bill": True,
    "biol": True,
    "bj": True,
    "bk": True,
    "bl": True,
    "bn": True,
    "both": True,
    "bottom": True,
    "bp": True,
    "br": True,
    "brief": True,
    "briefly": True,
    "bs": True,
    "bt": True,
    "bu": True,
    "but": True,
    "bx": True,
    "by": True,
    "c": True,
    "c1": True,
    "c2": True,
    "c3": True,
    "ca": True,
    "call": True,
    "came": True,
    "can": True,
    "cannot": True,
    "cant": True,
    "cause": True,
    "causes": True,
    "cc": True,
    "cd": True,
    "ce": True,
    "certain": True,
    "certainly": True,
    "cf": True,
    "cg": True,
    "ch": True,
    "changes": True,
    "ci": True,
    "cit": True,
    "cj": True,
    "cl": True,
    "clearly": True,
    "cm": True,
    "cmon": True,
    "cn": True,
    "co": True,
    "com": True,
    "come": True,
    "comes": True,
    "con": True,
    "concerning": True,
    "consequently": True,
    "consider": True,
    "considering": True,
    "contain": True,
    "containing": True,
    "contains": True,
    "corresponding": True,
    "could": True,
    "couldn": True,
    "couldnt": True,
    "course": True,
    "cp": True,
    "cq": True,
    "cr": True,
    "cry": True,
    "cs": True,
    "ct": True,
    "cu": True,
    "currently": True,
    "cv": True,
    "cx": True,
    "cy": True,
    "cz": True,
    "d": True,
    "d2": True,
    "da": True,
    "date": True,
    "dc": True,
    "dd": True,
    "de": True,
    "definitely": True,
    "describe": True,
    "described": True,
    "despite": True,
    "detail": True,
    "df": True,
    "di": True,
    "did": True,
    "didn": True,
    "didnt": True,
    "different": True,
    "dj": True,
    "dk": True,
    "dl": True,
    "do": True,
    "does": True,
    "doesn": True,
    "doesnt": True,
    "doing": True,
    "don": True,
    "done": True,
    "dont": True,
    "down": True,
    "downwards": True,
    "dp": True,
    "dr": True,
    "ds": True,
    "dt": True,
    "du": True,
    "due": True,
    "during": True,
    "dx": True,
    "dy": True,
    "e": True,
    "e2": True,
    "e3": True,
    "ea": True,
    "each": True,
    "ec": True,
    "ed": True,
    "edu": True,
    "ee": True,
    "ef": True,
    "effect": True,
    "eg": True,
    "ei": True,
    "eight": True,
    "eighty": True,
    "either": True,
    "ej": True,
    "el": True,
    "eleven": True,
    "else": True,
    "elsewhere": True,
    "em": True,
    "empty": True,
    "en": True,
    "end": True,
    "ending": True,
    "enough": True,
    "entirely": True,
    "eo": True,
    "ep": True,
    "eq": True,
    "er": True,
    "es": True,
    "especially": True,
    "est": True,
    "et": True,
    "et-al": True,
    "etc": True,
    "eu": True,
    "ev": True,
    "even": True,
    "ever": True,
    "every": True,
    "everybody": True,
    "everyone": True,
    "everything": True,
    "everywhere": True,
    "ex": True,
    "exactly": True,
    "example": True,
    "except": True,
    "ey": True,
    "f": True,
    "f2": True,
    "fa": True,
    "far": True,
    "fc": True,
    "few": True,
    "ff": True,
    "fi": True,
    "fifteen": True,
    "fifth": True,
    "fify": True,
    "fill": True,
    "find": True,
    "fire": True,
    "first": True,
    "five": True,
    "fix": True,
    "fj": True,
    "fl": True,
    "fn": True,
    "fo": True,
    "followed": True,
    "following": True,
    "follows": True,
    "for": True,
    "former": True,
    "formerly": True,
    "forth": True,
    "forty": True,
    "found": True,
    "four": True,
    "fr": True,
    "from": True,
    "front": True,
    "fs": True,
    "ft": True,
    "fu": True,
    "full": True,
    "further": True,
    "furthermore": True,
    "fy": True,
    "g": True,
    "ga": True,
    "gave": True,
    "ge": True,
    "get": True,
    "gets": True,
    "getting": True,
    "gi": True,
    "give": True,
    "given": True,
    "gives": True,
    "giving": True,
    "gj": True,
    "gl": True,
    "go": True,
    "goes": True,
    "going": True,
    "gone": True,
    "got": True,
    "gotten": True,
    "gr": True,
    "greetings": True,
    "gs": True,
    "gy": True,
    "h": True,
    "h2": True,
    "h3": True,
    "had": True,
    "hadn": True,
    "hadnt": True,
    "happens": True,
    "hardly": True,
    "has": True,
    "hasn": True,
    "hasnt": True,
    "have": True,
    "haven": True,
    "havent": True,
    "having": True,
    "he": True,
    "hed": True,
    "hell": True,
    "hello": True,
    "help": True,
    "hence": True,
    "her": True,
    "here": True,
    "hereafter": True,
    "hereby": True,
    "herein": True,
    "heres": True,
    "hereupon": True,
    "hers": True,
    "herself": True,
    "hes": True,
    "hh": True,
    "hi": True,
    "hid": True,
    "him": True,
    "himself": True,
    "his": True,
    "hither": True,
    "hj": True,
    "ho": True,
    "home": True,
    "hopefully": True,
    "how": True,
    "howbeit": True,
    "however": True,
    "hows": True,
    "hr": True,
    "hs": True,
    "http": True,
    "hu": True,
    "hundred": True,
    "hy": True,
    "i": True,
    "i2": True,
    "i3": True,
    "i4": True,
    "i6": True,
    "i7": True,
    "i8": True,
    "ia": True,
    "ib": True,
    "ibid": True,
    "ic": True,
    "id": True,
    "ie": True,
    "if": True,
    "ig": True,
    "ignored": True,
    "ih": True,
    "ii": True,
    "ij": True,
    "il": True,
    "ill": True,
    "im": True,
    "immediate": True,
    "immediately": True,
    "importance": True,
    "important": True,
    "in": True,
    "inasmuch": True,
    "inc": True,
    "indeed": True,
    "index": True,
    "indicate": True,
    "indicated": True,
    "indicates": True,
    "information": True,
    "inner": True,
    "insofar": True,
    "instead": True,
    "interest": True,
    "into": True,
    "invention": True,
    "inward": True,
    "io": True,
    "ip": True,
    "iq": True,
    "ir": True,
    "is": True,
    "isn": True,
    "isnt": True,
    "it": True,
    "itd": True,
    "itll": True,
    "its": True,
    "itself": True,
    "iv": True,
    "ive": True,
    "ix": True,
    "iy": True,
    "iz": True,
    "j": True,
    "jj": True,
    "jr": True,
    "js": True,
    "jt": True,
    "ju": True,
    "just": True,
    "k": True,
    "ke": True,
    "keep": True,
    "keeps": True,
    "kept": True,
    "kg": True,
    "kj": True,
    "km": True,
    "know": True,
    "known": True,
    "knows": True,
    "ko": True,
    "l": True,
    "l2": True,
    "la": True,
    "largely": True,
    "last": True,
    "lately": True,
    "later": True,
    "latter": True,
    "latterly": True,
    "lb": True,
    "lc": True,
    "le": True,
    "least": True,
    "les": True,
    "less": True,
    "lest": True,
    "let": True,
    "lets": True,
    "lf": True,
    "like": True,
    "liked": True,
    "likely": True,
    "line": True,
    "little": True,
    "lj": True,
    "ll": True,
    "ln": True,
    "lo": True,
    "look": True,
    "looking": True,
    "looks": True,
    "los": True,
    "lr": True,
    "ls": True,
    "lt": True,
    "ltd": True,
    "m": True,
    "m2": True,
    "ma": True,
    "made": True,
    "mainly": True,
    "make": True,
    "makes": True,
    "many": True,
    "may": True,
    "maybe": True,
    "me": True,
    "mean": True,
    "means": True,
    "meantime": True,
    "meanwhile": True,
    "merely": True,
    "mg": True,
    "might": True,
    "mightn": True,
    "mightnt": True,
    "mill": True,
    "million": True,
    "mine": True,
    "miss": True,
    "ml": True,
    "mn": True,
    "mo": True,
    "more": True,
    "moreover": True,
    "most": True,
    "mostly": True,
    "move": True,
    "mr": True,
    "mrs": True,
    "ms": True,
    "mt": True,
    "mu": True,
    "much": True,
    "mug": True,
    "must": True,
    "mustn": True,
    "mustnt": True,
    "my": True,
    "myself": True,
    "n": True,
    "n2": True,
    "na": True,
    "name": True,
    "namely": True,
    "nay": True,
    "nc": True,
    "nd": True,
    "ne": True,
    "near": True,
    "nearly": True,
    "necessarily": True,
    "necessary": True,
    "need": True,
    "needn": True,
    "neednt": True,
    "needs": True,
    "neither": True,
    "never": True,
    "nevertheless": True,
    "new": True,
    "next": True,
    "ng": True,
    "ni": True,
    "nine": True,
    "ninety": True,
    "nj": True,
    "nl": True,
    "nn": True,
    "no": True,
    "nobody": True,
    "non": True,
    "none": True,
    "nonetheless": True,
    "noone": True,
    "nor": True,
    "normally": True,
    "nos": True,
    "not": True,
    "noted": True,
    "nothing": True,
    "novel": True,
    "now": True,
    "nowhere": True,
    "nr": True,
    "ns": True,
    "nt": True,
    "ny": True,
    "o": True,
    "oa": True,
    "ob": True,
    "obtain": True,
    "obtained": True,
    "obviously": True,
    "oc": True,
    "od": True,
    "of": True,
    "off": True,
    "often": True,
    "og": True,
    "oh": True,
    "oi": True,
    "oj": True,
    "ok": True,
    "okay": True,
    "ol": True,
    "old": True,
    "om": True,
    "omitted": True,
    "on": True,
    "once": True,
    "one": True,
    "ones": True,
    "only": True,
    "onto": True,
    "oo": True,
    "op": True,
    "oq": True,
    "or": True,
    "ord": True,
    "os": True,
    "ot": True,
    "other": True,
    "others": True,
    "otherwise": True,
    "ou": True,
    "ought": True,
    "our": True,
    "ours": True,
    "ourselves": True,
    "out": True,
    "outside": True,
    "over": True,
    "overall": True,
    "ow": True,
    "owing": True,
    "own": True,
    "ox": True,
    "oz": True,
    "p": True,
    "p1": True,
    "p2": True,
    "p3": True,
    "page": True,
    "pagecount": True,
    "pages": True,
    "par": True,
    "part": True,
    "particular": True,
    "particularly": True,
    "pas": True,
    "past": True,
    "pc": True,
    "pd": True,
    "pe": True,
    "per": True,
    "perhaps": True,
    "pf": True,
    "ph": True,
    "pi": True,
    "pj": True,
    "pk": True,
    "pl": True,
    "placed": True,
    "please": True,
    "plus": True,
    "pm": True,
    "pn": True,
    "po": True,
    "poorly": True,
    "possible": True,
    "possibly": True,
    "potentially": True,
    "pp": True,
    "pq": True,
    "pr": True,
    "predominantly": True,
    "present": True,
    "presumably": True,
    "previously": True,
    "primarily": True,
    "probably": True,
    "promptly": True,
    "proud": True,
    "provides": True,
    "ps": True,
    "pt": True,
    "pu": True,
    "put": True,
    "py": True,
    "q": True,
    "qj": True,
    "qu": True,
    "que": True,
    "quickly": True,
    "quite": True,
    "qv": True,
    "r": True,
    "r2": True,
    "ra": True,
    "ran": True,
    "rather": True,
    "rc": True,
    "rd": True,
    "re": True,
    "readily": True,
    "really": True,
    "reasonably": True,
    "recent": True,
    "recently": True,
    "ref": True,
    "refs": True,
    "regarding": True,
    "regardless": True,
    "regards": True,
    "related": True,
    "relatively": True,
    "research": True,
    "research-articl": True,
    "respectively": True,
    "resulted": True,
    "resulting": True,
    "results": True,
    "rf": True,
    "rh": True,
    "ri": True,
    "right": True,
    "rj": True,
    "rl": True,
    "rm": True,
    "rn": True,
    "ro": True,
    "rq": True,
    "rr": True,
    "rs": True,
    "rt": True,
    "ru": True,
    "run": True,
    "rv": True,
    "ry": True,
    "s": True,
    "s2": True,
    "sa": True,
    "said": True,
    "same": True,
    "saw": True,
    "say": True,
    "saying": True,
    "says": True,
    "sc": True,
    "sd": True,
    "se": True,
    "sec": True,
    "second": True,
    "secondly": True,
    "section": True,
    "see": True,
    "seeing": True,
    "seem": True,
    "seemed": True,
    "seeming": True,
    "seems": True,
    "seen": True,
    "self": True,
    "selves": True,
    "sensible": True,
    "sent": True,
    "serious": True,
    "seriously": True,
    "seven": True,
    "several": True,
    "sf": True,
    "shall": True,
    "shan": True,
    "shant": True,
    "she": True,
    "shed": True,
    "shell": True,
    "shes": True,
    "should": True,
    "shouldn": True,
    "shouldnt": True,
    "shouldve": True,
    "show": True,
    "showed": True,
    "shown": True,
    "showns": True,
    "shows": True,
    "si": True,
    "side": True,
    "significant": True,
    "significantly": True,
    "similar": True,
    "similarly": True,
    "since": True,
    "sincere": True,
    "six": True,
    "sixty": True,
    "sj": True,
    "sl": True,
    "slightly": True,
    "sm": True,
    "sn": True,
    "so": True,
    "some": True,
    "somebody": True,
    "somehow": True,
    "someone": True,
    "somethan": True,
    "something": True,
    "sometime": True,
    "sometimes": True,
    "somewhat": True,
    "somewhere": True,
    "soon": True,
    "sorry": True,
    "sp": True,
    "specifically": True,
    "specified": True,
    "specify": True,
    "specifying": True,
    "sq": True,
    "sr": True,
    "ss": True,
    "st": True,
    "still": True,
    "stop": True,
    "strongly": True,
    "sub": True,
    "substantially": True,
    "successfully": True,
    "such": True,
    "sufficiently": True,
    "suggest": True,
    "sup": True,
    "sure": True,
    "sy": True,
    "system": True,
    "sz": True,
    "t": True,
    "t1": True,
    "t2": True,
    "t3": True,
    "take": True,
    "taken": True,
    "taking": True,
    "tb": True,
    "tc": True,
    "td": True,
    "te": True,
    "tell": True,
    "ten": True,
    "tends": True,
    "tf": True,
    "th": True,
    "than": True,
    "thank": True,
    "thanks": True,
    "thanx": True,
    "that": True,
    "thatll": True,
    "thats": True,
    "thatve": True,
    "the": True,
    "their": True,
    "theirs": True,
    "them": True,
    "themselves": True,
    "then": True,
    "thence": True,
    "there": True,
    "thereafter": True,
    "thereby": True,
    "thered": True,
    "therefore": True,
    "therein": True,
    "therell": True,
    "thereof": True,
    "therere": True,
    "theres": True,
    "thereto": True,
    "thereupon": True,
    "thereve": True,
    "these": True,
    "they": True,
    "theyd": True,
    "theyll": True,
    "theyre": True,
    "theyve": True,
    "thickv": True,
    "thin": True,
    "think": True,
    "third": True,
    "this": True,
    "thorough": True,
    "thoroughly": True,
    "those": True,
    "thou": True,
    "though": True,
    "thoughh": True,
    "thousand": True,
    "three": True,
    "throug": True,
    "through": True,
    "throughout": True,
    "thru": True,
    "thus": True,
    "ti": True,
    "til": True,
    "tip": True,
    "tj": True,
    "tl": True,
    "tm": True,
    "tn": True,
    "to": True,
    "together": True,
    "too": True,
    "took": True,
    "top": True,
    "toward": True,
    "towards": True,
    "tp": True,
    "tq": True,
    "tr": True,
    "tried": True,
    "tries": True,
    "truly": True,
    "try": True,
    "trying": True,
    "ts": True,
    "tt": True,
    "tv": True,
    "twelve": True,
    "twenty": True,
    "twice": True,
    "two": True,
    "tx": True,
    "u": True,
    "u201d": True,
    "ue": True,
    "ui": True,
    "uj": True,
    "uk": True,
    "um": True,
    "un": True,
    "under": True,
    "unfortunately": True,
    "unless": True,
    "unlike": True,
    "unlikely": True,
    "until": True,
    "unto": True,
    "uo": True,
    "up": True,
    "upon": True,
    "ups": True,
    "ur": True,
    "us": True,
    "use": True,
    "used": True,
    "useful": True,
    "usefully": True,
    "usefulness": True,
    "uses": True,
    "using": True,
    "usually": True,
    "ut": True,
    "v": True,
    "va": True,
    "value": True,
    "various": True,
    "vd": True,
    "ve": True,
    "very": True,
    "via": True,
    "viz": True,
    "vj": True,
    "vo": True,
    "vol": True,
    "vols": True,
    "volumtype": True,
    "vq": True,
    "vs": True,
    "vt": True,
    "vu": True,
    "w": True,
    "wa": True,
    "want": True,
    "wants": True,
    "was": True,
    "wasn": True,
    "wasnt": True,
    "way": True,
    "we": True,
    "wed": True,
    "welcome": True,
    "well": True,
    "well-b": True,
    "went": True,
    "were": True,
    "weren": True,
    "werent": True,
    "weve": True,
    "what": True,
    "whatever": True,
    "whatll": True,
    "whats": True,
    "when": True,
    "whence": True,
    "whenever": True,
    "whens": True,
    "where": True,
    "whereafter": True,
    "whereas": True,
    "whereby": True,
    "wherein": True,
    "wheres": True,
    "whereupon": True,
    "wherever": True,
    "whether": True,
    "which": True,
    "while": True,
    "whim": True,
    "whither": True,
    "who": True,
    "whod": True,
    "whoever": True,
    "whole": True,
    "wholl": True,
    "whom": True,
    "whomever": True,
    "whos": True,
    "whose": True,
    "why": True,
    "whys": True,
    "wi": True,
    "widely": True,
    "will": True,
    "willing": True,
    "wish": True,
    "with": True,
    "within": True,
    "without": True,
    "wo": True,
    "won": True,
    "wonder": True,
    "wont": True,
    "words": True,
    "world": True,
    "would": True,
    "wouldn": True,
    "wouldnt": True,
    "www": True,
    "x": True,
    "x1": True,
    "x2": True,
    "x3": True,
    "xf": True,
    "xi": True,
    "xj": True,
    "xk": True,
    "xl": True,
    "xn": True,
    "xo": True,
    "xs": True,
    "xt": True,
    "xv": True,
    "xx": True,
    "y": True,
    "y2": True,
    "yes": True,
    "yet": True,
    "yj": True,
    "yl": True,
    "you": True,
    "youd": True,
    "youll": True,
    "your": True,
    "youre": True,
    "yours": True,
    "yourself": True,
    "yourselves": True,
    "youve": True,
    "yr": True,
    "ys": True,
    "yt": True,
    "z": True,
    "zero": True,
    "zi": True,
    "zz": True,
}

In [93]:
# define cleaning functions

def normalize(text: str) -> str:
    """
    Normalize text from a pandas entry to ASCII-only lowercase characters. Hence, this removes Unicode characters with no ASCII
    equivalent (e.g., emojis and CJKs).

    This function may be used as a lambda in a map().
    
    # Parameters
    * text: Single string entry from a DataFrame column.

    # Returns
    ASCII-normalized text containing only lowercase letters.

    # Examples
    normalize("¿Cómo estás?")
    $ 'como estas?'

    normalize(" hahahaha HUY! Kamusta 😅 Mayaman $$$ ka na ba?") 
    $ ' hahahaha huy! kamusta  mayaman $$$ ka na ba?'
    """
    normalized = unicodedata.normalize('NFKD', text)
    ascii_text = normalized.encode('ascii', 'ignore').decode('ascii')

    return ascii_text.lower()


def rem_punctuation(text: str) -> str:
    """
    Removed the punctuations from a pandas entry. This function simply replaces all punctuation marks and special characters
    to the empty string. Hence, for symbols enclosed by whitespace, the whitespace are not collapsed to a single whitespace
    (for more information, see the examples).

    This function may be used as a lambda in a map().

    # Parameters
    * text: Single string entry from a DataFrame column.

    # Returns
    Text with the punctuation removed, or None if the result is empty or input invalid.

    # Examples
    rem_punctuation("this word $$ has two spaces after it!")
    $ 'this word  has two spaces after it'

    rem_punctuation("these!words@have$no%space")
    $ 'thesewordshavenospace'
    """
    return re.sub(f"[{re.escape(string.punctuation)}]", "", text)


def collapse_whitespace(text: str) -> str:
    """
    This collapses functions from a pandas entry. Here, collapsing means the transduction of all whitespace strings of any
    length to a whitespace string of unit length (e.g., "   " -> " "; formally " "+ -> " ").
    
    This function may be used as a lambda in a map().

    # Parameters
    * text: Single string entry from a DataFrame column.

    # Returns
    Text with the whitespaces collapsed.

    # Examples
    collapse_whitespace("  huh,  was.  that!!! ")
    $ 'huh, was. that!!!'
    """
    return re.sub(f" +", " ", text).strip()


def rem_stopwords(text: str, stopwords: dict[str, bool]) -> str:
    """
    This removes all stopwords from a pandas entry. For fast look up, `stopwords` is a dictionary with type [str,bool].
    This allows for constant time lookups as opposed to a linear search with a list.

    Strings detected as stopwords is replaced with the empty string "".

    This function may be used as a lambda in a map().

    # Parameters
    * text: Single string entry from a DataFrame column. ASSERT: the text must be all lowercase.
    * stopwords: stopword dictionary
    
    # Returns
    Text with the stopwords removed.

    # Examples
    rem_stopwords("he is a master", stopwords_lut)
    $ 'master'

    rem_stopwords("He is a master", stopwords_lut)
    $ AssertionError
    Remark. The substring "He" is not all lowercase.

    # Future
    If this function is too slow, we may implement `stopwords` as an 26-ary trie to achieve log-linear time.
    """
    assert text.islower(), "The value of `text` must be all lowercase" 
    
    words = text.split()
    filtered = [word for word in words if word not in stopwords]
    return " ".join(filtered)

AssertionError: The value of `text` must be all lowercase

# 3 preprocessing

> 🏗️ Perhaps swap S3 and S4. Refer to literature on what comes first.

This section discusses preprocessing steps for the cleaned data.

---
**References**
1. ()

In [20]:
pass

# 4 exploratory data analysis

This section discusses the exploratory data analysis conducted on the dataset after cleaning.

> Notes from Zhean: <br>
> From manual checking via OpenRefine, there are a total of 162972. `df.info()` should have the same result post-processing.
> Furthermore, there should be two columns, `clean_text` (which is a bit of a misnormer since it is still dirty) contains the Tweets (text data). The second column is the `category` which contains the sentiment of the Tweet and is a tribool (1 positive, 0 neutral or indeterminate, and -1 for negative).

---
**References**
1. ()